<a href="https://colab.research.google.com/github/ChiefGokhlayeh/aibv/blob/master/lab4/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labor 4 Deep Learning für die Solarzelleninspektion mit CNN

Im Kurs **(IE/DHL) Angewandte industrielle Bildverarbeitung S22** der **Hochschule für Angewandte Wissenschaften Hamburg**. Durchgeführt von:

* **Andreas Baulig**
* **Fabian Mahler**

Laborbetreuung:

* **Prof. Dr.-Ing. Dipl.-Kfm. J. Dahlkemper**

## Ziel

In diesem Versuch soll ein auf der Bilddatenbank ImageNet pre-trained CNN mit einem von Grund auf trainiertem CNN im Hinblick auf die Leistungsfähigkeit der Klassifizierung von Solarzellen auf Basis derer Elektrolumineszenz-Bilder verglichen werden.


## Vorbereitung

Zunächst wird geprüft, ob die für das Labor notwendigen Python Pakete installiert sind. Darunter fällt auch ein Test, ob TensorFlow mit GPU-Beschleunigung arbeiten kann.


In [ ]:
import io
import itertools
import os
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
from keras.applications import vgg16
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizer_v2.rmsprop import RMSProp
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [ ]:
# Check if GPU is present
if tf.test.gpu_device_name():
    print(f"Default GPU Device: {tf.test.gpu_device_name()}")
else:
    print("Please install GPU version of TF")

**NUR IN GOOGLE COLAB!**


In [ ]:
try:
    import google.colab  # noqa: F401 E402

    IN_COLAB = True
    print("Running in Google Colab.")
except ImportError:
    IN_COLAB = False
    print("Not running in Google Colab.")

In [ ]:
if IN_COLAB:
    from google.colab import drive  # noqa: E402

    drive.mount("/content/drive")

In [ ]:
def extract_dataset(fileobj, data_dir):
    with zipfile.ZipFile(fileobj) as z:
        for member in tqdm(
            z.infolist(), desc=f"Extracting to {os.path.normpath(data_dir)}"
        ):
            z.extract(member, data_dir)


if IN_COLAB:
    # In Google Colab we have to first download the zip file from the
    # repository.
    data_dir = "/content/data"
    dataset_source_url = "https://media.githubusercontent.com/media/ChiefGokhlayeh/aibv/master/data/multilabel_small.zip"
    with requests.get(dataset_source_url) as req:
        req.raise_for_status()

        extract_dataset(io.BytesIO(req.content), data_dir)

else:
    # Assume we are executing within the repository, so the zip file should be
    # available locally.
    data_dir = "../data"
    dataset_source_path = "../data/multilabel_small.zip"

    with open(dataset_source_path, "rb") as f:
        extract_dataset(f, data_dir)

multilabel_dir = os.path.join(data_dir, "multilabel")

## Bildtransformationen mit [`ImageDataGenerator`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

Keras Preprocessing bietet die Möglichkeit einen Bilderdatensatz mit synthetischen Transformationen der Originalbilder zu ergänzen. Die entsprechende Python Klasse dafür heißt [`ImageDataGenerator`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) und befindet sich im Modul `keras.preprocessing.image`. Die Bildgenerierung arbeitet in zwei Schritten. Zunächst wird der `ImageDataGenerator` mit einer Reihe von Transformationsparametern instantiiert. So wird festgelegt welche Veränderungen der `ImageDataGenerator` durchführen soll, um aus einem Originalbild ein neues synthetisches Bild zu erzeugen. Die Transformationen werden zufällig angewandt. Auch ist die Reihenfolge der Eingangsbilder randomisiert.

Aufgrund der speziellen Aufnahmebedingungen für Elektrolumineszenzbilder mit dem MBJ EL-Messplatz sind die Aufnahmen bereits in bestimmter Hinsicht normalisiert. Für die Solarzelleninspektion mit CNNs kommen folgende Transformationen zum Einsatz:

* `brightness_range` zur Simulation unterschiedlicher Leuchtintensitäten der Solarzelle.
* `height_shift_range` zur vertikalen Translation der Solarzellen um leichte Ausrichtungsfehler des EL-Messplatzes zu simulieren.
* `horizontal_flip` zur Horizontalen Spiegelung der Bilder. Dies Hilft, um Lokalätätsbiasse der Defekte im gegeben Trainingsdatensatz zu kompensieren.
* `shear_range` zur Scherung der Bilder, um leichte Montagefehler der Solarzelle im EL-Messplatz zu simulieren.
* `vertical_flip` zur Vertikalen Spiegelung der Bilder. Dies Hilft, um Lokalätätsbiasse der Defekte im gegeben Trainingsdatensatz zu kompensieren.
* `width_shift_range` zur horizontalen Translation der Solarzellen, um leichte Ausrichtungsfehler des EL-Messplatzes zu simulieren.

Mit dem vor-parametrierten `ImageDataGenerator` können nun die Originalbilder geladen werden. Um das Labeling beizubehalten, existiert die Methode [`ImageDataGenerator::flow_from_dataframe()`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_dataframe). Sie erwartet mindestens zwei Parameter: ein [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) mit den Bilddateinamen und zugehörigen Labels, und den Pfad einem Wurzelverzeichnis in dem die Dateien abliegen. Die im Datensatz gelieferte Label-CSV ist für Multi-Labeling ausgelegt und hat die Struktur:

|File|crack|darkarea|finger|
|----|-----|--------|------|
|0001.jpg|0|0|0|
|0002.jpg|0|0|0|
|0003.jpg|0|0|1|
|0004.jpg|0|0|1|
|...|...|...|...|
|0022.jpg|0|1|1|
|...|...|...|...|

Zu erkennen ist, dass für jedes Bild bis zu drei Labels vergeben werden können. Die Menge der Fehlerklassen lautet $L = \{ \mathrm{crack}, \mathrm{darkarea}, \mathrm{finger} \}$. Für jedes Bild gilt $y \subseteq L$, wobei die leere Menge $y = \emptyset$ eine Solarzelle ohne Defekte darstellt. 

### Siehe auch

* [API Dokumentation `tf.keras.preprocessing.image.ImageDataGenerator`  |  TensorFlow Core v2.9.1](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)
* [Image Augmentation for Deep Learning using Keras and Histogram Equalization | by Ryan Allred | Towards Data Science](https://towardsdatascience.com/image-augmentationfor-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085)

### Hinweis

Die verwendete `ImageDataGenerator` ist mit TensorFlow v2.9.0 als veraltet (deprecated) markiert worden. Zukünftige Implementierungen sollten die neuen APIs von [Keras Preprocessing](https://keras.io/api/preprocessing/image/) nutzen. Aufgrund der relativen Neuheit dieser Version setzt dieses Labor noch setzt auf TensorFlow v2.8.1.


In [ ]:
labels_df = pd.read_csv(os.path.join(multilabel_dir, "labels.csv"))

print(f"Dataset has a total of {len(labels_df)} items.")
labels_df.head()

In [ ]:
image_gen = ImageDataGenerator(
    brightness_range=(0.3, 1.1),
    cval=0,
    fill_mode="constant",
    height_shift_range=0.009,
    horizontal_flip=True,
    rotation_range=0,
    validation_split=0.2,
    shear_range=0.1,
    vertical_flip=True,
    width_shift_range=0.0085,
    preprocessing_function=vgg16.preprocess_input,
)

imggen_sample_size = 8
sample_batch_size = 16

label_names = labels_df.columns[1:].to_list()

flow_kwargs = {
    "batch_size": sample_batch_size,
    "class_mode": "raw",
    "color_mode": "rgb",
    "dataframe": labels_df,
    "directory": multilabel_dir,
    "shuffle": True,
    "target_size": (128, 128),
    "x_col": "File",
    "y_col": label_names,
}

train_iter = image_gen.flow_from_dataframe(
    subset="training",
    **flow_kwargs,
)
valid_iter = image_gen.flow_from_dataframe(
    subset="validation",
    **flow_kwargs,
)

In [ ]:
num_rows = 1
fig, axs = plt.subplots(num_rows, imggen_sample_size, squeeze=False, figsize=(20, 5))

for i in range(num_rows):
    batch, labels = train_iter.next()
    for j, (sample, label) in enumerate(
        itertools.islice(zip(batch, labels), imggen_sample_size)
    ):
        ax = axs[i, j]
        black = sample.min(axis=0)
        white = sample.max(axis=0)
        scale = (white - black) / 255
        img = (sample - black).astype(np.uint8)
        ax.imshow(img)
        labels_s = pd.Series(label, index=label_names)
        ax.set_title(", ".join(labels_s.index[labels_s == 1].to_list()) or "ok")

fig.tight_layout()

## Erzeuge CNN Modell

In [ ]:
base_model = vgg16.VGG16(
    include_top=False,
    classes=len(label_names),
    input_shape=flow_kwargs["target_size"] + (3,),
)
base_model.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_names), activation="sigmoid")(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    optimizer=RMSProp(learning_rate=10**-4),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=3),
    ModelCheckpoint(filepath="lab4_model.h5", monitor="val_loss", save_best_only=True),
]

hist = model.fit(
    train_iter,
    callbacks=callbacks,
    epochs=20,
    validation_data=valid_iter,
)
print(
    f"Model fitting completed with final validation accuracy={hist.history['val_accuracy'][-1]}"
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 6))

ax = axs[0]
ax.plot(hist.history["val_accuracy"])
ax.set_title("Validation Accuracy")
ax.set_xlabel("Epoch")
ax.grid(True)

ax = axs[1]
ax.plot(hist.history["val_loss"])
ax.set_title("Validation Loss")
ax.set_xlabel("Epoch")
ax.grid(True)

fig.tight_layout()

## **Meta**: Google Colab Code Formatierung und Aufräumen

Erlaubt es die Codezellen dieses Notebooks zu formatieren und etwaige Zellinhalte zu löschen. Besonderes letzteres sollte vor jedem `git commit` durchgeführt werden, um die `.ipynb` Datei klein zuhalten.

Nach dem Ausführen der Tools **nicht speichern**, stattdessen Seite mit `F5` neu laden. **Nur** für Verwendung in Google Colab gedacht. 

Inspiriert von https://stackoverflow.com/a/71001241.


In [ ]:
if IN_COLAB:
    %pip install black[jupyter] nbstripout
    !black '/content/drive/MyDrive/Colab Notebooks/lab4.ipynb'
    !nbstripout '/content/drive/MyDrive/Colab Notebooks/lab4.ipynb'